# test async
* [AsyncIO for the Working Python Developer](https://hackernoon.com/asyncio-for-the-working-python-developer-5c468e6e2e8e)
* asyncio = event loop + coroutine + future
  * [event loop](https://docs.python.org/dev/library/asyncio-eventloop.html)
    * 서로 다른 task들의 실행을 관리 & 분배
    * task들을 등록하고, task들 간의 제어 흐름을 분배하는 작업을 처리
  * [coroutine](https://docs.python.org/3.5/library/asyncio-task.html#coroutines)
    * 제너레이터와 유사한 특별한 함수
    * await keyword로 제어 흐름을 다시 event loop로 돌려줌
    * event loop를 통해 스케쥴되야 할 필요가 있으며, 이렇게 하기 위해 future의 일종인 task를 만듦
  * [future](https://docs.python.org/3.5/library/asyncio-task.html#future)
    * 실행이 될 수도, 안 될수도 있는 task의 결과를 표현하는 객체
    * 결과가 예외일 수도 있다
* [Rob Pike - ‘Concurrency Is Not Parallelism’](https://vimeo.com/49718712)
  * task들을 concurrent한 subtask들로 분해해야지만 parallelism을 할 수 있게 한다.
  * 이런 subtask들을 스케쥴링하는 일이 parallelism
* asyncio는 coroutine으로 정의되는 subtask들을 통해 코드를 구성할 수 있게 함
  * coroutine은 yield 지점을 포함하고 있는데, 다른 task들이 지연된다면 여기서 context switch가 발생할 수 있음
  * asyncio의 context switch는 한 coroutine에서 다른 coroutine으로 제어 흐름을 yield하는 event loop로 표현함

In [1]:
# %load 1-sync-async-execution-asyncio-await.py
import asyncio


async def foo():
    print('Running in foo')
    await asyncio.sleep(0)
    print('Explicit context switch to foo again')


async def bar():
    print('Explicit context to bar')
    await asyncio.sleep(0)
    print('Implicit context switch back to bar')


ioloop = asyncio.get_event_loop()
tasks = [ioloop.create_task(foo()), ioloop.create_task(bar())]
wait_tasks = asyncio.wait(tasks)
ioloop.run_until_complete(wait_tasks)
#ioloop.close()


Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


({<Task finished coro=<foo() done, defined at <ipython-input-1-ec8756bcd3df>:5> result=None>,
  <Task finished coro=<bar() done, defined at <ipython-input-1-ec8756bcd3df>:11> result=None>},
 set())

* foo(), bar()
  * coroutine. asyncio.sleep(0)을 사용해 non blocking으로 동작
  * await을 통해 coroutine이 event loop에 제어를 다시 넘겨줌
* create_task; coroutine은 다른 coroutine이나 task로 wrapped되어야 호출 가능
* wait; 완료하기 위해 두 개의 coroutine을 하나로 만듦
* run_until_complete; event loop를 이용해 스케쥴링
* coroutine이 yield하고 event loop가 실행을 위해 스케쥴된 다음 task로 context switch
  * foo -> bar -> foo -> bar

In [5]:
# %load 1b-cooperatively-scheduled-asyncio-await.py
import time
import asyncio

start = time.time()


def tic():
    return 'at %1.1f seconds' % (time.time() - start)


async def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('gr1 started work: {}'.format(tic()))
    await asyncio.sleep(2)
    print('gr1 ended work: {}'.format(tic()))


async def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('gr2 started work: {}'.format(tic()))
    await asyncio.sleep(2)
    print('gr2 Ended work: {}'.format(tic()))


async def gr3():
    print("Let's do some stuff while the coroutines are blocked, {}".format(tic()))
    await asyncio.sleep(1)
    print("Done!")


ioloop = asyncio.get_event_loop()
tasks = [
    ioloop.create_task(gr1()),
    ioloop.create_task(gr2()),
    ioloop.create_task(gr3())
]
ioloop.run_until_complete(asyncio.wait(tasks))
#ioloop.close()


gr1 started work: at 0.0 seconds
gr2 started work: at 0.0 seconds
Let's do some stuff while the coroutines are blocked, at 0.0 seconds
Done!
gr1 ended work: at 2.0 seconds
gr2 Ended work: at 2.0 seconds


({<Task finished coro=<gr1() done, defined at <ipython-input-5-a05bff2337c6>:12> result=None>,
  <Task finished coro=<gr2() done, defined at <ipython-input-5-a05bff2337c6>:19> result=None>,
  <Task finished coro=<gr3() done, defined at <ipython-input-5-a05bff2337c6>:26> result=None>},
 set())

* 2개의 blocking tasks, gr1 and gr2
* 2개를 실행하는 동안 gr3 asynchronous하게 작업
* 이벤트 루프는 싱글 스레드 코드가 concurrent하게 동작하는 실행을 관리하고 스케쥴
  * gr1과 gr2가 block되는 동안 gr3가 제어 흐름을 가져감

In [7]:
# %load 1c-determinism-sync-async-asyncio-await.py
import random
from time import sleep
import asyncio


def task(pid):
    """Synchronous non-deterministic task."""
    sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)


async def task_coro(pid):
    """Coroutine non-deterministic task"""
    await asyncio.sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)


def synchronous():
    for i in range(1, 10):
        task(i)


async def asynchronous():
    tasks = [asyncio.ensure_future(task_coro(i)) for i in range(1, 10)]
    await asyncio.wait(tasks)


print('Synchronous:')
synchronous()

ioloop = asyncio.get_event_loop()
print('Asynchronous:')
ioloop.run_until_complete(asynchronous())
#ioloop.close()


Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 3 done
Task 4 done
Task 9 done
Task 1 done
Task 2 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done


* Synchronous
  * 선형으로 생각하는데 익숙해짐
  * 서로 다른 실행시간을 소모하는 연속된 task들이 있는 경우, 호출 순서대로 실행
* Asynchronous
  * concurrency를 사용할 때는 스케쥴된 순서가 아니라 다른 순서로 task를 종료
  * `range(1, 10)`을 사용해 `synchronous()`와 동일한 순서로 스케쥴했지만 출력 결과는 완전히 다름
  * standard library의 `asyncio`를 사용할 때, 나머지 대부분은 blocking 기능만 제공한다는 점이 현재 주요 단점
    * 이 단점을 보완하기 위해 async 기능이 있는 여러가지 외부 library들을 사용(e.g. aiohttp)
  * [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html#module-concurrent.futures)를 사용해 thread나 process에서 blocking task를 wrap하고 `asyncio`를 사용하는 Future를 return할 수 있음
    * [github.com/yeraydiazdiaz/asyncio-ftwpd](https://github.com/yeraydiazdiaz/asyncio-ftwpd)

In [9]:
# %load 1d-async-fetch-from-server-asyncio-await.py
import time
import urllib.request
import asyncio
import aiohttp

URL = 'https://api.github.com/events'
MAX_CLIENTS = 3


def fetch_sync(pid):
    print('Fetch sync process {} started'.format(pid))
    start = time.time()
    response = urllib.request.urlopen(URL)
    datetime = response.getheader('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format(pid, datetime, time.time() - start))

    return datetime


async def fetch_async(pid):
    print('Fetch async process {} started'.format(pid))
    start = time.time()
    response = await aiohttp.request('GET', URL)
    datetime = response.headers.get('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format( pid, datetime, time.time() - start))

    response.close()
    return datetime


def synchronous():
    start = time.time()
    for i in range(1, MAX_CLIENTS + 1):
        fetch_sync(i)
    print("Process took: {:.2f} seconds".format(time.time() - start))


async def asynchronous():
    start = time.time()
    tasks = [asyncio.ensure_future( fetch_async(i)) for i in range(1, MAX_CLIENTS + 1)]
    await asyncio.wait(tasks)
    print("Process took: {:.2f} seconds".format(time.time() - start))


print('Synchronous:')
synchronous()

print('Asynchronous:')
ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()


Synchronous:
Fetch sync process 1 started
Process 1: Wed, 31 Jan 2018 04:45:26 GMT, took: 0.97 seconds
Fetch sync process 2 started
Process 2: Wed, 31 Jan 2018 04:45:27 GMT, took: 0.88 seconds
Fetch sync process 3 started
Process 3: Wed, 31 Jan 2018 04:45:28 GMT, took: 0.95 seconds
Process took: 2.81 seconds
Asynchronous:
Fetch async process 1 started
Fetch async process 2 started
Fetch async process 3 started


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x106d962b0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10732e7f0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10732ee48>


Process 1: Wed, 31 Jan 2018 04:45:29 GMT, took: 0.83 seconds
Process 2: Wed, 31 Jan 2018 04:45:29 GMT, took: 0.83 seconds
Process 3: Wed, 31 Jan 2018 04:45:29 GMT, took: 0.84 seconds
Process took: 0.86 seconds


* 가장 흔한 blocking task는 HTTP 서비스에서 데이터를 가져오는 작업
* [aiohttp](http://aiohttp.readthedocs.io/) library를 사용해 non-blocking HTTP 요청을 보내 github event API에서 Date response header만 가져오는 작업
  * 모든 request를 *동시에* 서비스에 보냄
  * 앞에서 이야기했듯이 각각의 request는 제어 흐름을 다음 request에 yield하고, 완료하면 return
  * code는 synchronous 버전과 매우 유사함

## Creating concurrency
* 이제까지, 'task 집합을 만들고 모두 종료하기를 기다리는 coroutine'에서 결과를 만들고 가져오는 하나의 메소드를 사용
* 다른 방식으로 실행하거나 결과를 가져오도록 coroutine을 스케쥴할 수도 있음

In [11]:
# %load 2a-async-fetch-from-server-as-completed-asyncio-await.py
import time
import random
import asyncio
import aiohttp

URL = 'https://api.github.com/events'
MAX_CLIENTS = 3


async def fetch_async(pid):
    start = time.time()
    sleepy_time = random.randint(2, 5)
    print('Fetch async process {} started, sleeping for {} seconds'.format( pid, sleepy_time))

    await asyncio.sleep(sleepy_time)

    response = await aiohttp.request('GET', URL)
    datetime = response.headers.get('Date')

    response.close()
    return 'Process {}: {}, took: {:.2f} seconds'.format( pid, datetime, time.time() - start)


async def asynchronous():
    start = time.time()
    futures = [fetch_async(i) for i in range(1, MAX_CLIENTS + 1)]
    for i, future in enumerate(asyncio.as_completed(futures)):
        result = await future
        print('{} {}'.format(">>" * (i + 1), result))

    print("Process took: {:.2f} seconds".format(time.time() - start))


ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()


Fetch async process 1 started, sleeping for 5 seconds
Fetch async process 2 started, sleeping for 2 seconds
Fetch async process 3 started, sleeping for 4 seconds


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x106d96208>


>> Process 2: Wed, 31 Jan 2018 05:32:03 GMT, took: 2.81 seconds


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10710ea20>


>>>> Process 3: Wed, 31 Jan 2018 05:32:05 GMT, took: 4.99 seconds


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10732e2e8>


>>>>>> Process 1: Wed, 31 Jan 2018 05:32:06 GMT, took: 5.80 seconds
Process took: 5.81 seconds


* 앞선 예제와 유사하지만, HTTP GET 요청이 도착하는대로 결과를 처리할 필요가 있다고 가정한 시나리오
* 모두 동시에 스케쥴, 결과는 요청이 도착한 순서대로. 
  * 앞의 예제와 다른 점은 *[as_completed](https://docs.python.org/dev/library/asyncio-task.html#asyncio.as_completed)*를 사용해 오는대로 completed future를 yield하는 iterator를 반환한 부분
  * *as_completed*와 *wait*은 원래 [*concurrent.futures*](https://docs.python.org/dev/library/concurrent.futures.html#module-functions)에서 유래

In [13]:
# %load 2b-fetch-first-ip-address-response-await.py
from collections import namedtuple
import time
import asyncio
from concurrent.futures import FIRST_COMPLETED
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'query')
)


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    response = await aiohttp.request('GET', service.url)
    json_response = await response.json()
    ip = json_response[service.ip_attr]

    response.close()
    return '{} finished with result: {}, took: {:.2f} seconds'.format( service.name, ip, time.time() - start)


async def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    done, pending = await asyncio.wait(futures, return_when=FIRST_COMPLETED)

    print(done.pop().result())


ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()


Fetching IP from ip-api
Fetching IP from ipify


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1072a4eb8>


ip-api finished with result: 211.56.96.151, took: 0.44 seconds


* IP address를 얻는 예제
  * 실행할 때 접속 가능한지는 확신할 수 없음
  * 하나씩 순서대로 점검하기를 원하지 않음
  * concurrent하게 request를 보내고 응답하는 첫 번째만 사용하려고 함
* [*wait*](https://docs.python.org/dev/library/asyncio-task.html#asyncio.wait)에 *return_when*이란 parameter가 있음
  * 지금까지는 단지 task들을 parallelise하려고 했기 때문에 wait가 반환하는 게 뭔지 신경쓰지 않았음
  * 이제부터 coroutine에서 결과를 가져오길 원하는데, future의 두 가지 집합, *done*과 *pending*을 사용할 수 있음
* 두 개의 task를 스케쥴하지만 첫 번째를 완료하면 loop를 닫고 두 번째는 pending으로 내버려둠
  * *Asyncio*는 그걸 버그로 가정하고 경고를 출력
  * event loop가 pending future를 상관하지 않게 알려줘야 함

## Future states
* future를 종료하면, 결과 메소드가 future의 결과를 반환
  * states
    * Pending, Cancelled; *InvalidStateError*
    * Cancelled; *CancelledError*
    * Done; 결과가 반환하거나 예외를 일으켰다는 뜻
  * *done*, *cancelled*; future가 해당 상태에 있는지 아닌지 boolean 반환
    * 저자가 running이라는 함수도 있다고 썼는데, 정작 python document에는 running이라는 함수는 없음
  * *cancel* 함수를 호출해 future를 취소할 수 있음
* 추가로 하고 싶은 작업이 있는 경우, done state가 됐을 때 동작하는 callback을 붙일 수 있음
* future의 결과나 예외를 수작업으로 정할 수 있는데, 보통 유닛 테스트 목적으로 사용함

In [15]:
# %load 2c-fetch-first-ip-address-response-no-warning-await.py
from collections import namedtuple
import time
import asyncio
from concurrent.futures import FIRST_COMPLETED
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'query')
)


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    response = await aiohttp.request('GET', service.url)
    json_response = await response.json()
    ip = json_response[service.ip_attr]

    response.close()
    return '{} finished with result: {}, took: {:.2f} seconds'.format( service.name, ip, time.time() - start)


async def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    done, pending = await asyncio.wait(futures, return_when=FIRST_COMPLETED)

    print(done.pop().result())

    for future in pending:
        future.cancel()


ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()


Task exception was never retrieved
future: <Task finished coro=<fetch_ip() done, defined at <ipython-input-13-64bff668c2af>:16> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Users/jun/programming/anaconda3/lib/python3.5/asyncio/tasks.py", line 241, in _step
    result = coro.throw(exc)
  File "<ipython-input-13-64bff668c2af>", line 20, in fetch_ip
    response = await aiohttp.request('GET', service.url)
  File "/Users/jun/programming/anaconda3/lib/python3.5/site-packages/aiohttp/client.py", line 732, in __await__
    return (yield from self._coro)
  File "/Users/jun/programming/anaconda3/lib/python3.5/site-packages/aiohttp/client.py", line 341, in _request
    break
  File "/Users/jun/programming/anaconda3/lib/python3.5/site-packages/aiohttp/helpers.py", line 727, in __exit__
    raise asyncio.TimeoutError from None
concurrent.futures._base.TimeoutError


Fetching IP from ipify
Fetching IP from ip-api


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x106d89f60>


ip-api finished with result: 211.56.96.151, took: 0.45 seconds


## Exception handling
* *Asyncio*는 concurrent code를 관리가능하고 가독성이 좋게 만드는 게 전부여서, 예외를 다루는 것도 명확해진다

In [20]:
# %load 3a-fetch-ip-addresses-fail-await.py
from collections import namedtuple
import time
import asyncio
import async_timeout
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'query'),
    Service('broken', 'http://no-way-this-is-going-to-work.com/json', 'ip')
)


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    try:
        with async_timeout.timeout(2):
            response = await aiohttp.request('GET', service.url)
    except:
        return '{} is unresponsive'.format(service.name)

    json_response = await response.json()
    ip = json_response[service.ip_attr]

    response.close()
    return '{} finished with result: {}, took: {:.2f} seconds'.format(service.name, ip, time.time() - start)


async def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    done, _ = await asyncio.wait(futures)

    for future in done:
        print(future.result())


ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()

Fetching IP from ip-api
Fetching IP from ipify
Fetching IP from broken
ipify is unresponsive
broken is unresponsive
ip-api is unresponsive


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x109ee3c18>


ipify is unresponsive
ip-api finished with result: 211.56.96.151, took: 0.25 seconds
broken is unresponsive


* 단순하게 *try...except* 사용하는 경우
* 원래대로 실행을 시켰더니 종료가 되지 않아 timeout을 넣어봄
  * async_timeout 참고; [docs.aiohttp.org/en/stable/client.html](https://docs.aiohttp.org/en/stable/client.html)

In [26]:
# %load 3b-fetch-ip-addresses-future-exceptions-await.py
from collections import namedtuple
import time
import asyncio
import async_timeout
import aiohttp
import traceback

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'this-is-not-an-attr'),
    Service('broken', 'http://no-way-this-is-going-to-work.com/json', 'ip')
)


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    try:
        with async_timeout.timeout(5):
            response = await aiohttp.request('GET', service.url)
    except:
        return '{} is unresponsive'.format(service.name)

    json_response = await response.json()
    ip = json_response[service.ip_attr]

    response.close()
    return '{} finished with result: {}, took: {:.2f} seconds'.format(
        service.name, ip, time.time() - start)


async def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    done, _ = await asyncio.wait(futures)

    for future in done:
        try:
            print(future.result())
        except:
            print("Unexpected error: {}".format(traceback.format_exc()))


ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()

Fetching IP from broken
Fetching IP from ipify
Fetching IP from ip-api


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10a22c0b8>


broken is unresponsive
Unexpected error: Traceback (most recent call last):
  File "<ipython-input-26-586d7894abb1>", line 42, in asynchronous
    print(future.result())
  File "/Users/jun/programming/anaconda3/lib/python3.5/asyncio/futures.py", line 274, in result
    raise self._exception
  File "/Users/jun/programming/anaconda3/lib/python3.5/asyncio/tasks.py", line 239, in _step
    result = coro.send(None)
  File "<ipython-input-26-586d7894abb1>", line 29, in fetch_ip
    ip = json_response[service.ip_attr]
KeyError: 'this-is-not-an-attr'

ipify is unresponsive


* 예상치 못한 exeption을 future의 결과로 처리
* ipify가 동작하지 않아 본문과는 다른 결과

In [28]:
# %load 3c-fetch-ip-addresses-ignore-exceptions-await.py
from collections import namedtuple
import time
import asyncio
import async_timeout
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'this-is-not-an-attr'),
    Service('borken', 'http://no-way-this-is-going-to-work.com/json', 'ip')
)


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    try:
        with async_timeout.timeout(1):
            response = await aiohttp.request('GET', service.url)
    except:
        print('{} is unresponsive'.format(service.name))
    else:
        json_response = await response.json()
        ip = json_response[service.ip_attr]

        response.close()
        print('{} finished with result: {}, took: {:.2f} seconds'.format(
            service.name, ip, time.time() - start))


async def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    await asyncio.wait(futures)  # intentionally ignore results


ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()

Fetching IP from ipify
Fetching IP from ip-api
Fetching IP from borken
borken is unresponsive


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10a2329b0>
Task exception was never retrieved
future: <Task finished coro=<fetch_ip() done, defined at <ipython-input-28-a38f2369c290>:17> exception=KeyError('this-is-not-an-attr',)>
Traceback (most recent call last):
  File "/Users/jun/programming/anaconda3/lib/python3.5/asyncio/tasks.py", line 239, in _step
    result = coro.send(None)
  File "<ipython-input-28-a38f2369c290>", line 28, in fetch_ip
    ip = json_response[service.ip_attr]
KeyError: 'this-is-not-an-attr'


ipify is unresponsive


* task를 스케쥴링하고 완료를 기다리지 않는 건 버그로 간주
* task를 스케쥴링하고 가능한 예외를 가져오지 않는 건 경고를 일으킴
* terminal에서 실행해야 차이(앞의 예제에서 asyncio의 출력은 제외)를 볼 수 있음

## Timeouts
* non blocking 호출에 timeout을 주고, 그 다음에 response를 보냄
* *wait*만 있으면 됨

In [ ]:
# %load 4a-timeout-with-wait-kwarg-await.py
import time
import random
import asyncio
import aiohttp
import argparse
from collections import namedtuple
from concurrent.futures import FIRST_COMPLETED

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'query'),
)

DEFAULT_TIMEOUT = 0.01


async def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))

    await asyncio.sleep(random.randint(1, 3) * 0.1)
    try:
        response = await aiohttp.request('GET', service.url)
    except:
        return '{} is unresponsive'.format(service.name)

    json_response = await response.json()
    ip = json_response[service.ip_attr]

    response.close()
    print('{} finished with result: {}, took: {:.2f} seconds'.format(
        service.name, ip, time.time() - start))
    return ip


async def asynchronous(timeout):
    response = {
        "message": "Result from asynchronous.",
        "ip": "not available"
    }

    futures = [fetch_ip(service) for service in SERVICES]
    done, pending = await asyncio.wait(
        futures, timeout=timeout, return_when=FIRST_COMPLETED)

    for future in pending:
        future.cancel()

    for future in done:
        response["ip"] = future.result()

    print(response)


parser = argparse.ArgumentParser()
parser.add_argument(
    '-t', '--timeout',
    help='Timeout to use, defaults to {}'.format(DEFAULT_TIMEOUT),
    default=DEFAULT_TIMEOUT, type=float)
args = parser.parse_args()

print("Using a {} timeout".format(args.timeout))
ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous(args.timeout))
ioloop.close()

* on terminal

    ```
    $ python3 4a-timeout-with-wait-kwarg-await.py
    Using a 0.01 timeout
    Fetching IP from ip-api
    Fetching IP from ipify
    {'message': 'Result from asynchronous.', 'ip': 'not available'}

    $ python3 4a-timeout-with-wait-kwarg-await.py -t 5
    Using a 5.0 timeout
    Fetching IP from ip-api
    Fetching IP from ipify
    Unclosed client session
    client_session: <aiohttp.client.ClientSession object at 0x109da5a20>
    ip-api finished with result: 211.56.96.151, took: 0.63 seconds
    {'message': 'Result from asynchronous.', 'ip': '211.56.96.151'}
    ```
* timeout argument는 *wait*에 사용